In [26]:
import pandas as pd

In [27]:
pd.set_option('display.max_columns', None)  # None muestra todas las columnas

# Establecer el ancho de las columnas para que no se trunque el contenido
pd.set_option('display.max_colwidth', None)

In [28]:
def get_unique_categories(path):
    dataset = pd.read_parquet(path)
    dataset['category2'] = dataset['category'].apply(lambda lst: pd.Series(lst).to_list())
    dataset['categories'] = dataset['category2'].apply(lambda x: ', '.join(x))
    data = dataset[['identificador','categories']]
    unique_categories = set()
    for categories in data['categories']:
        unique_categories.update(categories.split(', '))
    df = pd.DataFrame({'categories': list(unique_categories)})
    return df

In [29]:
california = get_unique_categories('CaliforniaFinalCorregido.parquet')
print(f'Largo: {len(california)}, tipo: {type(california)}')

Largo: 4006, tipo: <class 'pandas.core.frame.DataFrame'>


In [30]:
florida = get_unique_categories('FloridaFinalCorregido.parquet')
print(f'Largo: {len(florida)}, tipo: {type(florida)}')

Largo: 4102, tipo: <class 'pandas.core.frame.DataFrame'>


In [31]:
illinois = get_unique_categories('IllinoisFinalCorregido.parquet')
print(f'Largo: {len(illinois)}, tipo: {type(illinois)}')

Largo: 3284, tipo: <class 'pandas.core.frame.DataFrame'>


In [32]:
texas = get_unique_categories('TexasFinalCorregido.parquet')
print(f'Largo: {len(texas)}, tipo: {type(texas)}')

Largo: 2979, tipo: <class 'pandas.core.frame.DataFrame'>


In [33]:
new_york = get_unique_categories('NewYorkFinalCorregido.parquet')
print(f'Largo: {len(new_york)}, tipo: {type(new_york)}')

Largo: 2957, tipo: <class 'pandas.core.frame.DataFrame'>


In [34]:
total = pd.concat([california, florida, illinois, texas, new_york], ignore_index=True)
print(f'Largo: {len(total)}, tipo: {type(total)}')


Largo: 17328, tipo: <class 'pandas.core.frame.DataFrame'>


In [35]:
total.head()

,categories
0,
1,Osteopath
2,Sport tour agency
3,Bottled water supplier
4,Career guidance service


In [36]:
unicos = total.drop_duplicates()
print(f'Largo: {len(unicos)}, tipo: {type(unicos)}')

Largo: 4904, tipo: <class 'pandas.core.frame.DataFrame'>


In [37]:
unicos.to_csv('totales.csv', index=False)

In [16]:
unicos.head()

,categories
0,
1,School bus service
2,Car finance and loan company
3,Group accommodation
4,Commercial refrigeration


#chatgpt

In [15]:
#%pip install openai

In [38]:
import openai
openai.api_key  = 'TOKEN'

In [39]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

Fire protection equipment supplier
Corporate campus
Bank
Recreation
Driveshaft shop
Bookbinder
Airline ticket agency
Charity
News service
Wholesale bakery
Weather forecast service
Locks supplier
Estate appraiser
Pallet supplier
Cuban
Home Energy Auditors
Steel construction company
Model portfolio studio
Shanghainese
City administration
Foundry
Holding company


In [75]:
label = """Steel construction company

"""

In [76]:
prompt = f"""
I have the following label: '''{label}'''
Please analyze and tell me which of the following categories it best fits into. If the label is empty, please return "Uncategorized" and nothing else. You must choose one of these 14 categories and return the category name exactly as it appears in the list:
1. Restaurants
2. Shopping
3. Health and Beauty
4. Rental Services
5. Tourism
6. Entertainment
7. Health and Hospitals
8. Sports
9. Arts and Crafts
10. Events and Weddings
11. Automotive (Cars, Boats, Transportation, Planes)
12. Education and Learning
13. Veterinary and Pets
14. Gardening and Home Services
15. Technology, Networks, Electronics, and Engineering
16. Industry
17. Professional Services
18. Other

"""
response = get_completion(prompt)
print(response)

Industry


In [20]:
unicos_sample = unicos.sample(n=10, random_state=42)

#prueba con dataset

In [ ]:
#%pip install retry

In [12]:
from retry import retry

In [13]:
@retry(exceptions=openai.error.OpenAIError, tries=3, delay=2, backoff=2)
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [22]:
# Aplicar la función a la columna 'categories' y crear una nueva columna 'lines'
unicos_sample['lines'] = unicos_sample['categories'].apply(lambda label: get_completion(f"I have the following label: '{label}'\nPlease categorize it into one of the following options, or choose 'Uncategorized' if the label is empty: :\n1. Restaurants\n2. Shopping\n3. Health and Beauty\n4. Rental Services\n5. Tourism\n6. Entertainment\n7. Health and Hospitals\n8. Sports\n9. Arts and Crafts\n10. Events and Weddings\n11. Automotive (Cars, Boats, Transportation, Planes)\n12. Education and Learning\n13. Veterinary and Pets\n14. Miscellaneous"))


In [23]:
unicos_sample.lines.head()

11440              Uncategorized
3542                 2. Shopping
907      7. Health and Hospitals
2833          4. Rental Services
3106        3. Health and Beauty
Name: lines, dtype: object

#divido el dataframe en 10 para procesar por partes:

In [43]:
full_dataset = unicos.copy()

# Obtener el número de registros por conjunto más pequeño
num_subsets = 50
subset_size = len(full_dataset) // num_subsets

# Mezclar el dataset para garantizar una distribución aleatoria en cada subconjunto
shuffled_dataset = full_dataset.sample(frac=1, random_state=42)

# Dividir el dataset en subconjuntos más pequeños
subsets = [shuffled_dataset[i * subset_size : (i + 1) * subset_size] for i in range(num_subsets)]

# Guardar cada subconjunto como archivo CSV
for i, subset in enumerate(subsets):
    subset.to_csv(f'subset_{i + 1}.csv', index=False)
    print(f'Subconjunto {i + 1} guardado como subset_{i + 1}.csv')

Subconjunto 1 guardado como subset_1.csv
Subconjunto 2 guardado como subset_2.csv
Subconjunto 3 guardado como subset_3.csv
Subconjunto 4 guardado como subset_4.csv
Subconjunto 5 guardado como subset_5.csv
Subconjunto 6 guardado como subset_6.csv
Subconjunto 7 guardado como subset_7.csv
Subconjunto 8 guardado como subset_8.csv
Subconjunto 9 guardado como subset_9.csv
Subconjunto 10 guardado como subset_10.csv
Subconjunto 11 guardado como subset_11.csv
Subconjunto 12 guardado como subset_12.csv
Subconjunto 13 guardado como subset_13.csv
Subconjunto 14 guardado como subset_14.csv
Subconjunto 15 guardado como subset_15.csv
Subconjunto 16 guardado como subset_16.csv
Subconjunto 17 guardado como subset_17.csv
Subconjunto 18 guardado como subset_18.csv
Subconjunto 19 guardado como subset_19.csv
Subconjunto 20 guardado como subset_20.csv
Subconjunto 21 guardado como subset_21.csv
Subconjunto 22 guardado como subset_22.csv
Subconjunto 23 guardado como subset_23.csv
Subconjunto 24 guardado como 

In [15]:
import time

In [79]:
for i in range(40,50,1):
    data = pd.read_csv(f'subset_{i+1}.csv', header=None, names=['categories'])
    data = data.drop(0)
    data['lines'] = data['categories'].apply(lambda label: get_completion(f"I have the following label: '{label}'\nPlease categorize it into one of the following options: :\n1. Restaurants\n2. Shopping\n3. Health and Beauty\n4. Rental Services\n5. Tourism\n6. Entertainment\n7. Health and Hospitals\n8. Sports\n9. Arts and Crafts\n10. Events and Weddings\n11. Automotive (Cars, Boats, Transportation, Planes)\n12. Education and Learning\n13. Veterinary and Pets\n14. Miscellaneous\nOnly return the option selected. The return expected is for example 'N. Option', no more addtional text or information is required"))
    data.to_csv(f'subset_{i+1}_procesado.csv')
    time.sleep(30)

In [80]:
datax = pd.read_csv('subset_50_procesado.csv')
datax.head()

,Unnamed: 0,categories,lines
0,1,Aquarium shop,2. Shopping
1,2,Ravine,14. Miscellaneous
2,3,Refrigerator store,2. Shopping
3,4,Photo agency,14. Miscellaneous
4,5,Public transportation system,14. Miscellaneous


In [81]:
datax.lines.value_counts()

lines
14. Miscellaneous                                       27
9. Arts and Crafts                                      11
11. Automotive (Cars, Boats, Transportation, Planes)    10
2. Shopping                                              8
1. Restaurants                                           7
12. Education and Learning                               6
4. Rental Services                                       6
5. Tourism                                               5
8. Sports                                                5
3. Health and Beauty                                     4
7. Health and Hospitals                                  4
13. Veterinary and Pets                                  3
10. Events and Weddings                                  1
6. Entertainment                                         1
Name: count, dtype: int64

#combinar ahora todas las partes

In [85]:
# Lista para almacenar los DataFrames de los subconjuntos
subsets = []

# Cargar y concatenar los subconjuntos
num_subsets = 50
for i in range(num_subsets):
    subset = pd.read_csv(f'subset_{i + 1}_procesado.csv')
    subsets.append(subset)

# Concatenar los DataFrames en un solo DataFrame
combined_df = pd.concat(subsets, ignore_index=True, axis=0)

In [86]:
combined_df.head(10)

,Unnamed: 0,categories,lines
0,1,Military airport,"11. Automotive (Cars, Boats, Transportation, Planes)"
1,2,Mirror shop,2. Shopping
2,3,Foster care service,7. Health and Hospitals
3,4,Holiday home,4. Rental Services
4,5,Dental laboratory,3. Health and Beauty
5,6,Rehabilitation,7. Health and Hospitals
6,7,Mining equipment,14. Miscellaneous
7,8,Movie theater,6. Entertainment
8,9,American (Traditional),1. Restaurants
9,10,Ranches,14. Miscellaneous


In [87]:
combined_df.shape

(4900, 3)

In [88]:
combined_df.lines.value_counts()

lines
14. Miscellaneous                                                                          1028
7. Health and Hospitals                                                                     497
2. Shopping                                                                                 483
1. Restaurants                                                                              404
8. Sports                                                                                   360
11. Automotive (Cars, Boats, Transportation, Planes)                                        353
6. Entertainment                                                                            287
12. Education and Learning                                                                  282
3. Health and Beauty                                                                        268
4. Rental Services                                                                          240
5. Tourism                        

In [89]:
combined_df.to_csv("combined_df.csv")

#Ahora rechequeo las miscelaneas:

In [7]:
misc = pd.read_csv("combined_df_2.csv", sep=';')

In [8]:
misc.head()

,categories,lines
0,Military airport,"11. Automotive (Cars, Boats, Transportation, P..."
1,Mirror shop,2. Shopping
2,Foster care service,7. Health and Hospitals
3,Holiday home,4. Rental Services
4,Dental laboratory,3. Health and Beauty


In [98]:
solo_misc = misc[misc['lines']=='14. Miscellaneous']

In [99]:
solo_misc.shape

(1038, 2)

#vuelvo a partirlo para que lo procese

In [100]:
full_dataset = solo_misc.copy()

# Obtener el número de registros por conjunto más pequeño
num_subsets = 20
subset_size = len(full_dataset) // num_subsets

# Mezclar el dataset para garantizar una distribución aleatoria en cada subconjunto
shuffled_dataset = full_dataset.sample(frac=1, random_state=42)

# Dividir el dataset en subconjuntos más pequeños
subsets = [shuffled_dataset[i * subset_size : (i + 1) * subset_size] for i in range(num_subsets)]

# Guardar cada subconjunto como archivo CSV
for i, subset in enumerate(subsets):
    subset.to_csv(f'subset_misc_{i + 1}.csv', index=False)
    print(f'Subconjunto {i + 1} guardado como subset_{i + 1}.csv')

Subconjunto 1 guardado como subset_1.csv
Subconjunto 2 guardado como subset_2.csv
Subconjunto 3 guardado como subset_3.csv
Subconjunto 4 guardado como subset_4.csv
Subconjunto 5 guardado como subset_5.csv
Subconjunto 6 guardado como subset_6.csv
Subconjunto 7 guardado como subset_7.csv
Subconjunto 8 guardado como subset_8.csv
Subconjunto 9 guardado como subset_9.csv
Subconjunto 10 guardado como subset_10.csv
Subconjunto 11 guardado como subset_11.csv
Subconjunto 12 guardado como subset_12.csv
Subconjunto 13 guardado como subset_13.csv
Subconjunto 14 guardado como subset_14.csv
Subconjunto 15 guardado como subset_15.csv
Subconjunto 16 guardado como subset_16.csv
Subconjunto 17 guardado como subset_17.csv
Subconjunto 18 guardado como subset_18.csv
Subconjunto 19 guardado como subset_19.csv
Subconjunto 20 guardado como subset_20.csv


In [20]:
for i in range(20):
    data = pd.read_csv(f'subset_misc_{i+1}.csv')
    data['lines'] = data['categories'].apply(lambda label: get_completion(f"I have the following label: '{label}'\nPlease categorize it into one of the following options: :\n1. Restaurants\n2. Shopping\n3. Health and Beauty\n4. Rental Services\n5. Tourism\n6. Entertainment\n7. Health and Hospitals\n8. Sports\n9. Arts and Crafts\n10. Events and Weddings\n11. Automotive (Cars, Boats, Transportation, Planes)\n12. Education and Learning\n13. Veterinary and Pets\n14. Gardening and Home Services\n15. Technology, Networks, Electronics, and Engineering\n16. Industry\n17. Professional Services\n18. Other\nOnly return the option selected. The return expected is for example 'N. Option', no more addtional text or information is required"))
    data.to_csv(f'subset_misc_{i+1}_procesado.csv')
    time.sleep(30)

In [2]:
# Lista para almacenar los DataFrames de los subconjuntos
subsets = []

# Cargar y concatenar los subconjuntos
num_subsets = 20
for i in range(num_subsets):
    subset = pd.read_csv(f'subset_misc_{i + 1}_procesado.csv')
    subsets.append(subset)

# Concatenar los DataFrames en un solo DataFrame
combined_df_misc = pd.concat(subsets, ignore_index=True, axis=0)

In [4]:
combined_df_misc.shape

(1020, 3)

In [5]:
combined_df_misc.lines.value_counts()

lines
17. Professional Services                                                                                                                                      303
16. Industry                                                                                                                                                   205
18. Other                                                                                                                                                      184
14. Gardening and Home Services                                                                                                                                151
15. Technology, Networks, Electronics, and Engineering                                                                                                         103
5. Tourism                                                                                                                                                      26
8. Sports       

In [6]:
combined_df_misc.to_csv("combined_df_misc.csv")

#Ahora vamos a unir ambas partes, primero elinamos los miscellaneus y luego le agregamos la 2da parte 

In [10]:
inicial = pd.read_csv("combined_df_2.csv", sep=';')
inicial.lines.value_counts()

lines
14. Miscellaneous                                       1038
7. Health and Hospitals                                  497
2. Shopping                                              485
1. Restaurants                                           405
8. Sports                                                360
11. Automotive (Cars, Boats, Transportation, Planes)     353
6. Entertainment                                         288
12. Education and Learning                               283
3. Health and Beauty                                     268
4. Rental Services                                       240
5. Tourism                                               221
9. Arts and Crafts                                       217
13. Veterinary and Pets                                  210
10. Events and Weddings                                   34
Name: count, dtype: int64

In [12]:
inicial.shape

(4899, 2)

In [11]:
inicial.head()

,categories,lines
0,Military airport,"11. Automotive (Cars, Boats, Transportation, P..."
1,Mirror shop,2. Shopping
2,Foster care service,7. Health and Hospitals
3,Holiday home,4. Rental Services
4,Dental laboratory,3. Health and Beauty


In [13]:
inicial = inicial[inicial['lines'] != "14. Miscellaneous"]

In [14]:
inicial.shape

(3861, 2)

In [16]:
inicial.head(1)

,categories,lines
0,Military airport,"11. Automotive (Cars, Boats, Transportation, P..."


In [17]:
#cargo 2da parte:

In [19]:
segundo = pd.read_csv("combined_df_misc2.csv", sep=';')
segundo.head(1)

,categories,lines
0,Coal supplier,16. Industry


In [23]:
segundo.shape

(1020, 2)

#Ahora los uno

In [20]:
lines = pd.concat([inicial, segundo], ignore_index=True)

In [21]:
lines.head()

,categories,lines
0,Military airport,"11. Automotive (Cars, Boats, Transportation, P..."
1,Mirror shop,2. Shopping
2,Foster care service,7. Health and Hospitals
3,Holiday home,4. Rental Services
4,Dental laboratory,3. Health and Beauty


In [22]:
lines.shape

(4881, 2)

In [24]:
lines.lines.value_counts()

lines
7. Health and Hospitals                                   498
2. Shopping                                               485
1. Restaurants                                            411
8. Sports                                                 377
11. Automotive (Cars, Boats, Transportation, Planes)      354
17. Professional Services                                 305
6. Entertainment                                          299
12. Education and Learning                                283
3. Health and Beauty                                      270
5. Tourism                                                247
4. Rental Services                                        244
9. Arts and Crafts                                        218
13. Veterinary and Pets                                   212
16. Industry                                              205
18. Other                                                 185
14. Gardening and Home Services                           151
15

In [25]:
lines.to_csv('lines_final.csv', index=False)